# Тренировка нейросети с трансформером

Эту тренировку я сделела уже после того, как реализовала приложение,просто для того, чтоб не снимали баллы за её отсутствие. Всё работает хорошо и без тренировки)))))) 

In [3]:
import numpy as np
import pandas as pd
from pymystem3 import Mystem
import re


In [4]:
! pip install -U pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [5]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Данные

https://drive.google.com/file/d/15suP2_Uqi-Ni7FMHwzUbEpd6QcF5oina/view?usp=share_link - данные

In [6]:
! gdown -- 15suP2_Uqi-Ni7FMHwzUbEpd6QcF5oina

Downloading...
From: https://drive.google.com/uc?id=15suP2_Uqi-Ni7FMHwzUbEpd6QcF5oina
To: /content/test_rureviews - test_rureviews.tsv
100% 731k/731k [00:00<00:00, 39.7MB/s]


In [7]:
import csv

def find_delimiter(path):
    sniffer = csv.Sniffer()
    with open(path) as fp:
        delimiter = sniffer.sniff(fp.read(5000)).delimiter
    return delimiter

In [8]:
df = pd.read_csv("test_rureviews - test_rureviews.tsv", sep='\t')

In [9]:
df.columns =['review', 'sentiment']

In [10]:
df.head(10)

,review,sentiment
0,"очень странно сел, большой размера на 2. слишк...",neutral
1,Заказ был сделан 20.10 курьер принес 14.11. По...,neutral
2,холодная,neutral
3,"Доставка - 3 недели в Ростов-на-Дону, цена 385...",positive
4,"куртка размер 6XL соответствует 46 размеру, пр...",negative
5,очень долго шел товар!!!,neutral
6,"Отличная толстовка!!! на свой 44 заказала xl, ...",positive
7,"на 42-44 можно смело заказывать xs, качество х...",positive
8,"ждала три месяца а заказ так и не пришел, прод...",negative
9,Заказ пришёл за 10 дней в Москву. Все швы ровн...,positive


In [14]:
idx_to_label = {0: "neutral", 1: "toxic"}
label_to_idx = {"neutral":0, "neautral":0, "skip":0, "speech":0, "negative":1, "positive":0}

In [15]:
sent_ind = []
for i in range(len(df)):
    sent_ind.append(label_to_idx[df['sentiment'][i]])

df['label'] = sent_ind

In [16]:
df.head()

,review,sentiment,label
0,"очень странно сел, большой размера на 2. слишк...",neutral,0
1,Заказ был сделан 20.10 курьер принес 14.11. По...,neutral,0
2,холодная,neutral,0
3,"Доставка - 3 недели в Ростов-на-Дону, цена 385...",positive,0
4,"куртка размер 6XL соответствует 46 размеру, пр...",negative,1


In [17]:
text_df = df.drop('sentiment', axis=1)

In [18]:
text_df.head(10)

,review,label
0,"очень странно сел, большой размера на 2. слишк...",0
1,Заказ был сделан 20.10 курьер принес 14.11. По...,0
2,холодная,0
3,"Доставка - 3 недели в Ростов-на-Дону, цена 385...",0
4,"куртка размер 6XL соответствует 46 размеру, пр...",1
5,очень долго шел товар!!!,0
6,"Отличная толстовка!!! на свой 44 заказала xl, ...",0
7,"на 42-44 можно смело заказывать xs, качество х...",0
8,"ждала три месяца а заказ так и не пришел, прод...",1
9,Заказ пришёл за 10 дней в Москву. Все швы ровн...,0


In [19]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.7 MB/s eta 0:00:00


In [20]:
from transformers import pipeline

In [21]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
from transformers import pipeline

clf = pipeline(
    task = 'sentiment-analysis', 
    model = 'SkolkovoInstitute/russian_toxicity_classifier')

text = ['Сегодня хорошая погода',
    	'Что за урод там ходит?', 'всё супер']

clf(text)


[{'label': 'neutral', 'score': 0.9986207485198975},
 {'label': 'toxic', 'score': 0.9709195494651794},
 {'label': 'neutral', 'score': 0.9997263550758362}]

In [23]:
import datasets
import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer)

train, test = train_test_split(text_df, test_size=0.3)
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)


tokenizer = AutoTokenizer.from_pretrained(
    'SkolkovoInstitute/russian_toxicity_classifier')

def tokenize_function(examples):
	return tokenizer(examples['review'], padding='max_length', truncation=True)

tokenized_train = train.map(tokenize_function)
tokenized_test = test.map(tokenize_function)

model = AutoModelForSequenceClassification.from_pretrained(
	'SkolkovoInstitute/russian_toxicity_classifier',
	num_labels=2)

training_args = TrainingArguments(
	output_dir = 'test_trainer_log',
	evaluation_strategy = 'epoch',
	per_device_train_batch_size = 6,
	per_device_eval_batch_size = 6,
	num_train_epochs = 5,
	report_to='none')


metric = evaluate.load('f1')
def compute_metrics(eval_pred):
	logits, labels = eval_pred
	predictions = np.argmax(logits, axis=-1)
	return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
	model = model,
	args = training_args,
	train_dataset = tokenized_train,
	eval_dataset = tokenized_test,
	compute_metrics = compute_metrics)

trainer.train()


save_directory = './pt_save_pretrained'

model.save_pretrained(save_directory)

Map:   0%|          | 0/2099 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,No log,0.421732,0.753719
2,0.503100,0.510128,0.728223
3,0.321400,0.787594,0.721429
4,0.321400,0.946078,0.735552
5,0.115700,0.975999,0.754209


In [32]:
from transformers import pipeline

clf = pipeline(
    task = 'sentiment-analysis', 
    model = model.to('cpu'),
    tokenizer = tokenizer)
text = ['Сегодня хорошая погода',
    	'Что за урод там ходит?', 'всё супер']


clf(text)

[{'label': 'neutral', 'score': 0.9956596493721008},
 {'label': 'toxic', 'score': 0.973989725112915},
 {'label': 'neutral', 'score': 0.999627947807312}]